In [ ]:
import numpy as np
from scipy.integrate import solve_bvp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Fungsi potensial
def potential(r, m):
    rand = np.random.rand(len(r))
    V = -5 * m + 0.002 * rand - 0.001
    return V

# Fungsi untuk menghitung persamaan Schrödinger
def schrodinger_eq(r, y, E, m):
    R, dR_dr = y
    d2R_dr2 = -2 * (E - potential(r, m)) * R - (2 / r) * dR_dr
    return np.vstack([dR_dr, d2R_dr2])

# Kondisi batas untuk fungsi gelombang
def boundary_conditions(ya, yb):
    return np.array([ya[0], yb[0]])

# Menghitung energi level menggunakan metode numerik
def calculate_energy_levels(m, r_max, n_levels=50):
    r = np.linspace(0.01, r_max, 1000)  # mulai dari 0.01 untuk menghindari pembagian dengan nol
    E_guess = np.linspace(-5 * m - 0.001, 0, n_levels)
    energy_levels = []

    for E in E_guess:
        R_guess = np.zeros((2, r.size))
        R_guess[0, 0] = 1e-5
        R_guess[1, 0] = 1e-5

        sol = solve_bvp(lambda r, y: schrodinger_eq(r, y, E, m), boundary_conditions, r, R_guess)
        if sol.success:
            energy_levels.append(E)

    return np.array(energy_levels)


In [ ]:
np.random.seed(42)  # Untuk reproduktibilitas
m_values = np.linspace(1, 10, 20)  # Misalkan massa isotop dari 1 hingga 10
radii = 1.2 * m_values**(1/3)
energy_data = np.array([calculate_energy_levels(m, r_max) for m, r_max in zip(m_values, radii)])
m_labels = m_values

In [ ]:
# Memisahkan data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(energy_data, m_labels, test_size=0.2, random_state=42)

# Standardisasi data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Membangun model ANN
model = Sequential()
model.add(Dense(64, input_dim=50, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Mengompilasi model
model.compile(loss='mean_squared_error', optimizer='adam')

# Melatih model
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_split=0.2)


Epoch 1/100
2/2 [==============================] - 2s 242ms/step - loss: 30.7586 - val_loss: 24.7002
Epoch 2/100
2/2 [==============================] - 0s 37ms/step - loss: 26.2299 - val_loss: 22.7638
Epoch 3/100
2/2 [==============================] - 0s 38ms/step - loss: 22.4872 - val_loss: 21.0132
Epoch 4/100
2/2 [==============================] - 0s 36ms/step - loss: 19.4164 - val_loss: 19.3392
Epoch 5/100
2/2 [==============================] - 0s 37ms/step - loss: 16.4346 - val_loss: 17.6222
Epoch 6/100
2/2 [==============================] - 0s 50ms/step - loss: 13.9681 - val_loss: 16.0284
Epoch 7/100
2/2 [==============================] - 0s 51ms/step - loss: 11.6359 - val_loss: 14.5668
Epoch 8/100
2/2 [==============================] - 0s 38ms/step - loss: 9.9416 - val_loss: 13.2353
Epoch 9/100
2/2 [==============================] - 0s 38ms/step - loss: 8.6394 - val_loss: 12.0582
Epoch 10/100
2/2 [==============================] - 0s 37ms/step - loss: 7.8565 - val_loss: 11.0969
E

In [ ]:
# Memprediksi massa isotop untuk data yang diberikan
given_energy_levels = np.array([0, 0.0001, -0.0004, 0.0012, -0.0003, 0.0010, 0.0009, 0.0003, 0.0001, 0.0019,
                                0.0018, 0.0006, 0.0012, 0.0016, 0.0023, 0.0028, 0.0032, 0.0030, 0.0031, 0.0038,
                                0.0031, 0.0045, 0.0046, 0.0052, 0.0045, 0.0042, 0.0062, 0.0060, 0.0067, 0.0068,
                                0.0081, 0.0069, 0.0084, 0.0078, 0.0081, 0.0097, 0.0101, 0.0111, 0.0102, 0.0107,
                                0.0124, 0.0133, 0.0128, 0.0141, 0.0148, 0.0155, 0.0151, 0.0162, 0.0167, 0.0181]).reshape(1, -1)

given_energy_levels_scaled = scaler.transform(given_energy_levels)

predicted_mass = model.predict(given_energy_levels_scaled)

# Menampilkan hasil prediksi
print(f"Predicted mass of the isotop: {predicted_mass[0][0]}")


1/1 [==============================] - 0s 106ms/step
Predicted mass of the isotop: 3.9546756744384766
